In [ ]:
# Gerekli kütüphaneleri içeri alıyoruz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Uyarıları kapatmak için (temiz görünsün diye)
import warnings
warnings.filterwarnings('ignore')

print("Kütüphaneler yüklendi!")